# Introduction

These are some notes I am taking as I learn about MCP. Much of it I assume will come from the MCP documentation.


[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to LLMs.

<img src="static_blog_imgs/mcp_arch.png" width="80%">

- MCP Hosts: programs like Claude Desktop, Cursor IDE, AI tools, etc
- MCP Clients: Protocol clients that maintain 1:1 connection with the servers
    - the client lives within the host
- MCP servers: lightweight programs that each expose specific capabilities through the MCP standard
- Local Data Sources: your computers files, databases, and services that MCP servers can securely access
- Remote Services: external systems available over the internet (e.g. through APIs) that MCP servers can connect to


# Quickstart

The quickstart is broken out in different paths:
    - for server developers
    - for client developers
    - for Claude desktop users

## Claude Desktop Users

### Filesystem MCP Server
I'm going to start with adding my first MCP server to Claude Desktop. I followed the guide [here](https://modelcontextprotocol.io/quickstart/user).
You end up editing the file `~/Library/Application\ Support/Claude/claude_desktop_config.json`

Add this content to the file (Make sure to replace username with your computer’s username.)
The paths should point to valid directories that you want Claude to be able to access and modify.

```
{
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/Users/christopher/Desktop",
        "/Users/christopher/Downloads"
      ]
    }
  }
}
```

*This configuration file tells Claude for Desktop which MCP servers to start up every time you start the application. In this case, we have added one server called “filesystem” that will use the Node npx command to install and run **@modelcontextprotocol/server-filesystem**. This server, described [here](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem), will let you access your file system in Claude for Desktop.*

<img src="static_blog_imgs/file_system_added.png" width="30%"> <img src="static_blog_imgs/file_system_tools.png" width="30%">


I asked claude to make a text file on my desktop with some ascii art. It figures it can use the file system tools to do this.
It asks permission for each step and you have the option to allow always.

<img src="static_blog_imgs/file_claude_ask1.png" width="30%"> <img src="static_blog_imgs/file_claude_ask2png.png" width="30%"> <img src="static_blog_imgs/ex_ascii_art.png" width="30%">


I can see how I could achieve the same thing with traditional function/cool calling, but this did feel nice in that it "just worked".
Throughout these notes I'm sure Im going to keep coming back to the idea of MCP tools VS Function calling. There is a X post I like [here](https://x.com/akshay_pachaar/status/1929153463822569952) with some nice visuals explaining the differences between the two. I understand that MCP is much more than traditional function calling. I do like the idea of having standardized tools that I can just connect stuff to. 

Let's see if I can find another useful MCP server to connect to Claude Desktop.







### Sequential Thinking MCP Server

Another interesting MCP server is the sequential thinking server. 
I came across it in the [MCP docs](https://modelcontextprotocol.io/examples).

[Sequential Thinking MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/sequentialthinking)

After adding it, my `claude_desktop_config.json` file looks like this:

```json
{
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/Users/christopher/Desktop",
        "/Users/christopher/Downloads"
      ]
    },
    "sequential-thinking": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-sequential-thinking"
      ]
    }
  }
}

```

This mcp server allows the LLM to have chain of thought sequential reasoning, I guess. 

<img src="static_blog_imgs/seq_thought.png" width="30%">

I wonder how useful this is now that we have really good reasoning models.

## Cursor IDE as a Host

### Remote GitHub MCP Server

I use Cursor on a daily basis but don't use MCPs. Let's see if I can connect
the GitHub Remote MCP server to Cursor.

The official GitHub MCP server is [here](https://github.com/github/github-mcp-server).

They recommend using the remote server in the docs [here](https://github.com/github/github-mcp-server/blob/main/docs/installation-guides/install-cursor.md) when
connecting to Cursor.  While Cursor supports OAuth for some MCP servers, the GitHub server currently requires a Personal Access Token.

The global MCP configuration for Cursor is located at `~/.cursor/mcp.json`.


```
{
  "mcpServers": {
    "github": {
      "url": "https://api.githubcopilot.com/mcp/",
      "headers": {
        "Authorization": "Bearer YOUR_GITHUB_PAT"
      }
    }
  }
}
```



<img src="static_blog_imgs/cursor_gh_mcp.png" width="30%">

If you are running the GitHub MCP server locally in docker you can pass a read only [flag]https://github.com/github/github-mcp-server?tab=readme-ov-file#read-only-mode).
But when running through the remote server, I guess that is not an option. By the way, you can add MCP servers with one click to Cursor [here](https://docs.cursor.com/tools/mcp).

I set up a fine grained personal access token and that is how I am controlling the amount of access the MCP server has.

<img src="static_blog_imgs/pat_cursor.png" width="100%">

For example, I gave it read/write to one repo for only code, commit statuses, and pull requests. But it cant create repos, delete them, do all kinds of other things.
If you try and use a tool outside of that, it will give you an error. So I **highly** recommend setting up a fine grained personal access token and use that token for the remote MCP server.

I can ask for example what were the most recent PR changes:

<img src="static_blog_imgs/mcp_pr_changes.png" width="50%">

Previous to testing this MCP server, I always just told the Cursor LLM agent
to use the GitHub CLI to make pull requests, commits, etc. So at this point,
I'm not sure what all the value I would personally get from using the MCP server is (since the GitHub CLI was proving quite useful).









# Resources

- [good blog post from Vercel on MCP](https://vercel.com/blog/model-context-protocol-mcp-explained)